In [12]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '../src/')

In [33]:
import numpy as np

import torch
import torch.nn.functional as F

from util import masked_softmax

In [19]:
samples = 4
maxlen = 6

In [24]:
logits = torch.randn((samples, maxlen))
logits

tensor([[ 0.2672,  0.3516, -2.5833, -0.1929, -0.2811,  0.0372],
        [-0.3425, -1.1083,  0.2950, -1.3571, -1.2215,  1.8443],
        [-0.4433,  1.1551, -0.1436, -0.3647,  1.0059, -0.0839],
        [ 0.4208, -1.2898, -0.8294,  1.2627,  0.0663,  0.4260]])

In [28]:
lengths = torch.tensor([4, 6, 1, 3])
lengths

tensor([4, 6, 1, 3])

In [26]:
mask = torch.arange(maxlen)[None, :] < lengths[:, None]
mask.int()

tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0]], dtype=torch.int32)

In [27]:
probs = masked_softmax(logits, mask, dim=-1)
probs

tensor([[0.3601, 0.3918, 0.0208, 0.2273, 0.0000, 0.0000],
        [0.0767, 0.0357, 0.1451, 0.0278, 0.0318, 0.6830],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6816, 0.1232, 0.1952, 0.0000, 0.0000, 0.0000]])

In [30]:
logits * mask

tensor([[ 0.2672,  0.3516, -2.5833, -0.1929, -0.0000,  0.0000],
        [-0.3425, -1.1083,  0.2950, -1.3571, -1.2215,  1.8443],
        [-0.4433,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
        [ 0.4208, -1.2898, -0.8294,  0.0000,  0.0000,  0.0000]])

In [37]:
def test_masked_softmax(logits, mask):
    for dim in range(len(logits.shape)):
        result = masked_softmax(logits, mask, dim=dim)
        
        correct_result = F.softmax(logits * mask, dim=dim)
        correct_result = correct_result * mask
        correct_result = correct_result / (correct_result.sum(dim=dim, keepdim=True) + 1e-13)
        
        assert np.allclose(result.numpy(), correct_result.numpy())

test_masked_softmax(logits, mask)

tensor([[0.3124, 0.7013, 0.0407, 0.7621, 0.0000, 0.0000],
        [0.1698, 0.1629, 0.7241, 0.2379, 1.0000, 1.0000],
        [0.1535, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3643, 0.1358, 0.2352, 0.0000, 0.0000, 0.0000]])
tensor([[0.3124, 0.7013, 0.0407, 0.7621, 0.0000, 0.0000],
        [0.1698, 0.1629, 0.7241, 0.2379, 1.0000, 1.0000],
        [0.1535, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3643, 0.1358, 0.2352, 0.0000, 0.0000, 0.0000]])
tensor([[0.3601, 0.3918, 0.0208, 0.2273, 0.0000, 0.0000],
        [0.0767, 0.0357, 0.1451, 0.0278, 0.0318, 0.6830],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6816, 0.1232, 0.1952, 0.0000, 0.0000, 0.0000]])
tensor([[0.3601, 0.3918, 0.0208, 0.2273, 0.0000, 0.0000],
        [0.0767, 0.0357, 0.1451, 0.0278, 0.0318, 0.6830],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6816, 0.1232, 0.1952, 0.0000, 0.0000, 0.0000]])
